<a href="https://colab.research.google.com/github/Rahul626-stack/Capstone_Project/blob/main/Model_1_Capstone_CLEANED_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh panel --quiet

import pandas as pd
import pathway as pw
import datetime
import panel as pn
pn.extension()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Load dataset
df = pd.read_csv("dataset.csv")

# Combine date and time into Timestamp
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# Sort by time
df = df.sort_values('Timestamp').reset_index(drop=True)

# Map vehicle weights
veh_weight_map = {"car": 1.0, "bike": 0.8, "truck": 1.5 ,"cycle": 0.5}
df['veh_weight'] = df['VehicleType'].map(veh_weight_map).fillna(1.0)

# Select columns for streaming
stream_df = df[[
    "Timestamp", "SystemCodeNumber", "Capacity",
    "Occupancy", "IsSpecialDay", "veh_weight"
]]

# Save to CSV
stream_df.to_csv("parking_stream.csv", index=False)


In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp:         str
    SystemCodeNumber:  str
    Capacity:          float
    Occupancy:         float
    IsSpecialDay:      int
    veh_weight:        float

source = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=500  # Simulated rows/second
)

# Parse datetime
source = source.with_columns(
    t = source.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S")
)


In [ ]:
alpha, beta = 5.0, 0.5

hourly_price = (
    source
    .windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        window_end = pw.this._pw_window_end,
        occupancy  = pw.reducers.max(pw.this.Occupancy),
        capacity   = pw.reducers.max(pw.this.Capacity),
        is_special = pw.reducers.max(pw.this.IsSpecialDay),
        veh_weight = pw.reducers.max(pw.this.veh_weight),
        system_id  = pw.reducers.max(pw.this.SystemCodeNumber),
    )
    .with_columns(
        Price = 10
              + alpha
              * (pw.this.occupancy / pw.this.capacity)
              * pw.this.veh_weight
              * (1 + beta * pw.this.is_special)
    )
    .select(
        SystemCodeNumber = pw.this.system_id,
        Timestamp        = pw.this.window_end,
        veh_weight       = pw.this.veh_weight,
        Price            = pw.this.Price
    )
)



In [ ]:
pw.io.csv.write(
    table=hourly_price,
    filename="half_hourly_price_output.csv"
)
import warnings
from beartype.roar import BeartypeDecorHintPep585DeprecationWarning

warnings.filterwarnings("ignore", category=BeartypeDecorHintPep585DeprecationWarning)


/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [ ]:
import panel as pn
import bokeh.plotting as bp
from bokeh.models import ColumnDataSource

pn.extension()

#  Streaming plot function
def price_plotter(source: ColumnDataSource):
    fig = bp.figure(
        height=400,
        width=800,
        title="Real-Time Parking Price",
        x_axis_type="datetime",
        x_axis_label="Timestamp",
        y_axis_label="Price"
    )

    # Line + small scatter dots (same color)
    fig.line("Timestamp", "Price", source=source, line_width=2, color="navy")
    fig.scatter("Timestamp", "Price", source=source, size=4, color="navy")

    return fig

# Attach plot to your hourly_price Pathway table
viz = hourly_price.plot(price_plotter, sorting_col="Timestamp")

#  Display using Panel
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
pw.run()


Output()

In [ ]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot, column
from bokeh.models import ColumnDataSource, HoverTool, Legend
from bokeh.palettes import Category10

# Enable Bokeh output in notebook/Colab
output_notebook()

# Load your Pathway output
df = pd.read_csv("half_hourly_price_output.csv")
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

#  Map known vehicle weights to types
weight_to_type = {
    0.5: "Cycle",
    0.8: "Bike",
    1.0: "Car",
    1.5: "Truck"
}
df["veh_bin"] = df["veh_weight"].map(weight_to_type)

#  Color and plot setup
palette = Category10[10]
color_map = {}
plots = []
legend_items = {}

#  Create one plot per vehicle type
for bin_name in ["Cycle", "Bike", "Car", "Truck"]:
    p = figure(
        width=500,
        height=350,
        title=f"Vehicle Type: {bin_name}",
        x_axis_type="datetime",
        x_axis_label="Time",
        y_axis_label="Price"
    )

    # Add hover tool
    p.add_tools(HoverTool(
        tooltips=[("Time", "@x{%F %H:%M}"), ("Price", "@y")],
        formatters={"@x": "datetime"},
        mode="vline"
    ))

    # Filter data for current vehicle type
    sub_df = df[df["veh_bin"] == bin_name]

    # Line per system
    for syscode, sys_df in sub_df.groupby("SystemCodeNumber"):
        color = color_map.setdefault(syscode, palette[len(color_map) % len(palette)])
        sys_df = sys_df.sort_values("Timestamp")

        source = ColumnDataSource(data={
            "x": sys_df["Timestamp"],
            "y": sys_df["Price"]
        })

        line = p.line("x", "y", source=source, line_width=2, color=color)

        # Only collect one legend entry per system
        if syscode not in legend_items:
            legend_items[syscode] = (f"Lot {syscode}", [line])

    plots.append(p)

# Create separate legend figure
legend_fig = figure(
    width=800,
    height=120,
    title="Legend: Parking Lot (SystemCodeNumber)",
    toolbar_location=None,
    outline_line_color=None,
    min_border=0,
    x_range=(0, 1),
    y_range=(0, 1)
)
legend_fig.axis.visible = False
legend_fig.grid.visible = False

# Add all legend items
legend = Legend(items=list(legend_items.values()), location="center", orientation="horizontal", label_text_font_size="9pt")
legend_fig.add_layout(legend, 'center')

#  Layout: 2x2 grid of subplots + legend below
grid = gridplot([plots[i:i+2] for i in range(0, len(plots), 2)])
show(column(grid, legend_fig))
